# 1. 패키지 설치

In [1]:
%pip install python-dotenv langchain langchain-upstage langchain-community langchain-text-splitters langchain-chroma

Note: you may need to restart the kernel to use updated packages.


# 2. Knowledge Base 구성을 위한 데이터 생성

- [RecursiveCharacterTextSplitter](https://python.langchain.com/v0.2/docs/how_to/recursive_text_splitter/)를 활용한 데이터 chunking
    - split 된 데이터 chunk를 Large Language Model(LLM)에게 전달하면 토큰 절약 가능
    - 비용 감소와 답변 생성시간 감소의 효과
    - LangChain에서 다양한 [TextSplitter](https://python.langchain.com/v0.2/docs/how_to/#text-splitters)들을 제공
- `chunk_size` 는 split 된 chunk의 최대 크기
- `chunk_overlap`은 앞 뒤로 나뉘어진 chunk들이 얼마나 겹쳐도 되는지 지정

In [2]:
%pip install docx2txt

Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./Journey_mate.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [6]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

# 환경변수를 불러옴
load_dotenv()

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = UpstageEmbeddings(model="solar-embedding-1-large")

In [7]:
from langchain_chroma import Chroma

# 데이터를 처음 저장할 때 
database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name='journey-mate', persist_directory="./chroma")

# 이미 저장된 데이터를 사용할 때 
# database = Chroma(collection_name='chroma-tax', persist_directory="./chroma", embedding_function=embedding)

# 3. 답변 생성을 위한 Retrieval

- `Chroma`에 저장한 데이터를 유사도 검색(`similarity_search()`)를 활용해서 가져옴

In [8]:
query = '당진 근처 여행지를 추천해주세요'

# `k` 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
retrieved_docs = database.similarity_search(query, k=4)

In [9]:
retrieved_docs

[Document(metadata={'source': './Journey_mate.docx'}, page_content='버거킹 당진읍내점,충남 당진시 당진중앙1로 95,충남 당진시 읍내동 513-2,매운 맛으로 유명한 주꾸미와 칼국수를 맛볼 수 있는 인기 맛집\n\n삽교천 시외버스터미널,충남 당진시 신평면 삽교천길 103,충남 당진시 신평면 운정리 181-6,청주 인근의 편안한 휴식과 다양한 편의시설을 제공하는 휴게소\n\n삽교호 전망대 종합 어 시장,충남 당진시 신평면 삽교천3길 93,충남 당진시 신평면 운정리 187-4,신선한 해산물을 구매할 수 있는 현지 어시장\n\n삽교 배수갑문,,충남 당진시 신평면 운정리 901,아름다운 바다 경관을 즐길 수 있는 횟집\n\n삽교호 바다공원,충남 당진시 신평면 삽교천3길 57,충남 당진시 신평면 운정리 214-5,평화로운 해변가에서 편안한 휴식을 즐길 수 있는 펜션\n\n온양온천 역 1호선,충남 아산시 온천대로 1496,충남 아산시 온천동 56-9,매운 맛으로 유명한 주꾸미와 칼국수를 맛볼 수 있는 인기 맛집\n\n모텔 미라보,충남 부여군 부여읍 정림로 47-28,충남 부여군 부여읍 구아리 294-1,청주 인근의 편안한 휴식과 다양한 편의시설을 제공하는 휴게소\n\n구드래 황토지,충남 부여군 규암면 백제문로 30,충남 부여군 규암면 규암리 61-1,신선한 해산물을 구매할 수 있는 현지 어시장\n\n여행은 나랑,충남 부여군 부여읍 중앙로13번길 25,충남 부여군 부여읍 구아리 153-6,아름다운 바다 경관을 즐길 수 있는 횟집\n\n스플라스 리솜,충남 예산군 덕산면 온천단지3로 45-7,충남 예산군 덕산면 사동리 362,평화로운 해변가에서 편안한 휴식을 즐길 수 있는 펜션\n\n공주알밤 휴게쉼터,충남 공주시 정안면 차령로 2792-2,충남 공주시 정안면 북계리 305-3,매운 맛으로 유명한 주꾸미와 칼국수를 맛볼 수 있는 인기 맛집\n\n천안박물관,,충남 천안시 동남구 삼룡동 260-11,청주 인근의 편안한 휴식과 다

# 4. Augmentation을 위한 Prompt 활용

- Retrieval된 데이터는 LangChain에서 제공하는 프롬프트(`"rlm/rag-prompt"`) 사용

In [9]:
%pip install -U langchain langchainhub --quiet

Note: you may need to restart the kernel to use updated packages.


In [10]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage()

In [11]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

c:\Users\ehd01\anaconda3\envs\test-env3\lib\site-packages\langsmith\client.py:333: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
c:\Users\ehd01\anaconda3\envs\test-env3\lib\site-packages\langsmith\client.py:5434: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


# 5. 답변 생성

- [RetrievalQA](https://docs.smith.langchain.com/old/cookbook/hub-examples/retrieval-qa-chain)를 통해 LLM에 전달
    - `RetrievalQA`는 [create_retrieval_chain](https://python.langchain.com/v0.2/docs/how_to/qa_sources/#using-create_retrieval_chain)으로 대체됨
    - 실제 ChatBot 구현 시 `create_retrieval_chain`으로 변경하는 과정을 볼 수 있음

In [12]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [13]:
ai_message = qa_chain({"query": query})

C:\Users\ehd01\AppData\Local\Temp\ipykernel_20268\3455095564.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  ai_message = qa_chain({"query": query})


In [14]:
# 강의에서는 위처럼 진행하지만 업데이트된 LangChain 문법은 `.invoke()` 활용을 권장
ai_message = qa_chain.invoke({"query": query})

In [15]:
ai_message

{'query': '당진 근처 여행지를 추천해주세요',
 'result': '당진 근처 여행지로는 삽교호 관광지, 왜목마을, 필경사, 아미미술관, 덕산온천, 태안해안국립공원, 대천해수욕장, 서산해미읍성, 수덕사 등이 추천됩니다.'}